In [9]:
"""
Grover's Search Benchmark Program - Qiskit
"""

import sys
import time

from qiskit import IBMQ, Aer, QuantumCircuit, transpile
import strangeworks as sw
import numpy as np
import pytket
from strangeworks.qiskit import StrangeworksProvider
from strangeworks.braket import get_backends, run_circuit, get_circuit_results
from strangeworks.qiskit.jobs.strangeworksjob import StrangeworksJob
import pytket.extensions.qiskit as pyqi #strangeworks-qiskit
import pytket.extensions.braket as pybra #strangeworks-braket
# import pytket.extensions.ionq as ionq



In [10]:
print(sw.__version__)
sw.authenticate(username="auchoi", api_key="") #read from text file to hide api
provider = StrangeworksProvider()

0.2.11


In [11]:
# provider = IBMQ.load_account()
# backend = provider.get_backend('ibmq_lima')
"""Basic Qiskit circuit with no TKET adjustment"""

print(provider.backends()) # to see a list of backends you can use from IBM. Provider is set to IBMQ
qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)
qc.measure([0,1],[0,1])
# meas = QuantumCircuit(2,2)
# meas.measure([0,1],[0,1])

regularBackend = provider.get_backend('ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_qasm_simulator')
# drawing = qc.compose(meas)
job = regularBackend.run(qc, shots=50)
# job = regularBackend.run(drawing, shots=50)
print(job.job_id())
print(job.status())
#
# pullJob = StrangeworksJob.get(job.job_id())
# result = pullJob.result()
# print(result.to_dict())
result = job.result()
counts = result.get_counts()

print(counts)



[aws.aws-managed.Aspen-10, aws.aws-managed.Aspen-11, aws.aws-managed.Aspen-8, aws.aws-managed.Aspen-9, aws.aws-managed.Aspen-M-1, aws.aws-managed.IonQ Device, aws.aws-managed.SV1, aws.aws-managed.dm1, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_qasm_simulator, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_armonk, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_santiago, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_bogota, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibmq_jakarta, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibm_lagos, ibm.caholder.ibm-q-deloitte.internal.quantum-guild.ibm_perth, ibm.caholder.ibm-q-deloitte.internal.deloitte-us-quan.ibmq_qasm_simulator, ibm.caholder.ibm-q-deloitte.internal.deloitte-us-quan.ibmq_armonk, ibm.caholder.ibm-q-deloitte.internal.deloitte-us-quan.ibmq_santiago, ibm.caholder.ibm-q-deloitte.internal.deloitte-us-quan.ibmq_bogota, ibm.caholder.ibm-q-deloitte.internal.deloitte-us-quan.ibmq_jakarta

In [12]:
"""Qiskit to TKET conversion """

tketCirc = pyqi.qiskit_to_tk(qc)
tketCirc.measure_all()
print(type(tketCirc))


<class 'pytket._tket.circuit.Circuit'>


In [13]:
"""Running on TKET backends"""

tketQiskitBackend = pyqi.AerBackend()
tket_compiled_circ = tketQiskitBackend.get_compiled_circuit(tketCirc)
tket_handle = tketQiskitBackend.process_circuit(tketCirc,n_shots=50)
tket_counts = tketQiskitBackend.get_result((tket_handle)).get_counts()
print(tket_counts)

Counter({(0, 0): 26, (1, 1): 24})


In [14]:
"""Braket Implementation"""

print(get_backends()) #this method is from the strangeworks-braket package
braketCircuit = pybra.tk_to_braket(tketCirc)
braketJob = run_circuit(braketCircuit,backend='aws.SV1', shots=50)
print(braketJob)
braketResult = get_circuit_results(braketJob)
print(braketResult)

['aws.Aspen-10', 'aws.Aspen-11', 'aws.Aspen-8', 'aws.Aspen-9', 'aws.Aspen-M-1', 'aws.IonQ Device', 'aws.Lucy', 'aws.SV1', 'aws.dm1']
GateModelQuantumTaskResult(task_metadata=TaskMetadata(braketSchemaHeader=BraketSchemaHeader(name='braket.task_result.task_metadata', version='1'), id='arn:aws:braket:us-west-1:498168020561:quantum-task/b65802a9-6d65-4a5a-9a7e-565bd581306a', shots=50, deviceId='arn:aws:braket:::device/quantum-simulator/amazon/sv1', deviceParameters=GateModelSimulatorDeviceParameters(braketSchemaHeader=BraketSchemaHeader(name='braket.device_schema.simulators.gate_model_simulator_device_parameters', version='1'), paradigmParameters=GateModelParameters(braketSchemaHeader=BraketSchemaHeader(name='braket.device_schema.gate_model_parameters', version='1'), qubitCount=2, disableQubitRewiring=False)), createdAt='2022-05-12T17:08:41.142Z', endedAt='2022-05-12T17:08:42.774Z', status='COMPLETED', failureReason=None), additional_metadata=AdditionalMetadata(action=Program(braketSchemaH